In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "None"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [2]:
import sys
sys.path.append("..")
%load_ext autoreload

import jax.numpy as jnp
from jax import random

In [5]:
%autoreload 2
from configs.EI_leaky_final import get_config

config = get_config()
config.seed = 3

In [6]:
from models import NeuralOperator, ENN, _create_encoder_arch, _create_decoder_arch, _create_epitrain_arch, _create_epiprior_arch
encoder = _create_encoder_arch(config.encoder_arch)
decoder = _create_decoder_arch(config.decoder_arch)
base_net = NeuralOperator(encoder, decoder)
epi_train = _create_epitrain_arch(config.epitrain_arch)
epi_prior = _create_epiprior_arch(config.epiprior_arch)
arch = ENN(base_net, epi_train, epi_prior, config.ensemble_size, config.scale, config.output_activation)
u = jnp.ones(config.input_dim)
y = jnp.ones(config.query_dim)
z = jnp.ones(config.ensemble_size)
key = random.PRNGKey(config.seed)
print(arch.tabulate(key, u, y, z))


                                  ENN Summary                                   
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs        ┃ outputs       ┃ params       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│               │ ENN           │ - float32[4]  │ float32[1]    │              │
│               │               │ - float32[2]  │               │              │
│               │               │ - float32[16] │               │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ base_net      │ NeuralOperat… │ - float32[4]  │ - float32[1]  │              │
│               │               │ - float32[2]  │ - float32[64] │              │
│               │               │               │ - float32[64] │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ base_net/enc… │ MlpEncode